In [1]:
import sympy as sp 
from sympy import exp, Integral, oo, Rational, pi, sqrt, factorial, factorial2

In [2]:
A, B, R, g, V, lm, ga, gf, x, V_dot_g, alp, u1, u2, mu1, mu2 = sp.symbols(f'A B R g V \lambda g_a g_f x \Lambda alpha u_1 u_2 \mu_1 \mu_2', positive=True, real = True)
l, k, m = sp.symbols(f'l k m', negative=False, integer = True)

Implementation of the angular integrals, given by Eqs. (100) and (101) in the draft.
 * When dom>0, the fragmentative integral is performed
 * When dom<0, the restitutive integral is performed
 * When dom=0, the aggregative integral, e.g. no domain restriction is performed

In [3]:
def angular_integral(m,p,q=0,dom=0):
    if (q != 0) and (q != 1):
        return "q has to be either 0 or 1"
    I_mpq0 = (-1)**(p+q)*(V_dot_g)**q*2*pi*g**(m+p-q)/(m+p+q+1)
    Frag_dom = (1-(gf/g)**(m+p+q+1))*sp.Heaviside(g-gf)
    Rest_dom = sp.Heaviside(gf-g)+sp.Heaviside(g-gf)*(gf/g)**(m+p+q+1)
    if dom==0:
        res = I_mpq0
    elif dom > 0:
        res = I_mpq0 * Frag_dom
    else:
        res = I_mpq0 * Rest_dom
    return res

An example of the angular integral
$$
    I^{m,p,q}_{\hat{n},r}=\int d\hat{n}\Theta(-\vec{g}\cdot\hat{n})
    \vert\vec{g}\cdot\hat{n}\vert^m(\vec{g}\cdot\hat{n})^p(\vec{V}\cdot\hat{n})^q
    \Theta(g_f^2-(\vec{g}\cdot\hat{n})^2),
$$
for specific values of $m,p,q$ is shown

In [4]:
angular_integral(m=1,p=2,q=0,dom=-1)

pi*g**3*(Heaviside(-g + g_f) + g_f**4*Heaviside(g - g_f)/g**4)/2

Implementation of the center of mass velocity integrals, given by Eq. (134) in the draft.

In [5]:
def CoM_integral(l,q=0):
    if (q != 0) and (q != 1):
        return "q has to be either 0 or 1"
    C = (2/R)**q*(pi/A)**Rational(3,2)*exp(lm**2)/(A**l)
    T1 = (factorial2(2*l+1)/(2**(l+1)))**q
    T2 = 0
    for k in range(0, l+1):
        F1 = factorial(2*l+1)/(factorial(2*k)*factorial(2*l-2*k+1))
        F2 = factorial2(2*k-1)/(2**k)
        F3 = lm**(2*l-2*k)
        F4 = (lm**2*(l+1)/(l-k+1)-Rational(1,2))**q
        T2 += F1*F2*F3*F4
    res = C*(T1+T2)
    return res.simplify()

In [6]:
CoM_integral(l=1,q=0)

pi**(3/2)*(\lambda**2 + 5/2)*exp(\lambda**2)/A**(5/2)